In [ ]:

import os
os.environ["WANDB_DISABLED"] = "true"

!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes

from unsloth import FastLanguageModel
import torch
from datasets import load_dataset

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# LoRA
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    outputs      = examples["output"]
    texts = []
    for instruction, output in zip(instructions, outputs):
        text = alpaca_prompt.format(instruction, "", output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }


dataset = load_dataset("json", data_files="train_dataset.json", split="train")
dataset = dataset.map(formatting_prompts_func, batched = True,)


from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

print("Starting Training...")
trainer_stats = trainer.train()
print("Training Complete!")

model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")
print("Model saved to 'lora_model' folder.")

In [ ]:
from unsloth import FastLanguageModel
import torch
import json
from tqdm import tqdm
import os

# 1. מעבר למצב 'הסקה' (Inference Mode)
# אנחנו משתמשים במודל שכבר נמצא בזיכרון מהשלב הקודם!
FastLanguageModel.for_inference(model)

# 2. הגדרת הפרומפט והפונקציה
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
"""

def generate_response(context, question):
    # הכנת הפרומפט
    inputs = tokenizer(
        [
            alpaca_prompt.format(
                "You are a banking regulation compliance assistant. Based strictly on the provided context, answer the user's question in JSON format. Required fields: \"answer\" (Yes/No), \"citation\" (verbatim quote), \"explanation\" (concise justification).",
                f"Context: {context}\nQuestion: {question}",
                "",
            )
        ], return_tensors = "pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
    decoded = tokenizer.batch_decode(outputs)

    response = decoded[0].split("### Response:\n")[-1].replace(tokenizer.eos_token, "").strip()
    return response


if not os.path.exists("test_dataset.json"):
    print("⚠️ STOP! You must upload 'test_dataset.json' to the files sidebar first.")
else:

    print("Loading test data...")
    with open("test_dataset.json", "r", encoding="utf-8") as f:
        test_data = json.load(f)

    results = []
    print(f"🚀 Starting evaluation on {len(test_data)} examples...")


    for item in tqdm(test_data):
        full_text = item['instruction']
        try:

            if "Context: " in full_text and "\nQuestion: " in full_text:
                context_part = full_text.split("Context: ")[1].split("\nQuestion: ")[0]
                question_part = full_text.split("\nQuestion: ")[1]
            else:
                continue


            prediction = generate_response(context_part, question_part)

            results.append({
                "question": question_part,
                "context": context_part,
                "true_answer": item['output'],
                "model_prediction": prediction
            })
        except Exception as e:
            print(f"Skipped an item due to error: {e}")
            continue


    with open("evaluation_results.json", "w", encoding="utf-8") as f:
        json.dump(results, f, indent=4)

    print("\n Evaluation Done! Saved to 'evaluation_results.json'")
    print("Refresh the files sidebar to see the result file.")

Loading test data...
🚀 Starting evaluation on 50 examples...


100%|██████████| 50/50 [04:12<00:00,  5.06s/it]


✅ Evaluation Done! Saved to 'evaluation_results.json'
Refresh the files sidebar to see the result file.
